<h2> Taller 9 - Cinemática Diferencial</h2>

<br>Requiere que los archivos 'sim.py', 'simConst.py', 'remoteapi.dll' estén alojados en la misma carpeta que este cuaderno de Jupyter.
<br> Desde CoppeliaSim, abrir la escena robotPlanar.ttt


In [ ]:
# importamos las librerías necesarias
import sim
import sympy as sp

###  Conectamos con Sim y obtenemos los manejadores
Utilizamos las funciones del API Remoto de VREP.
<br> Para más detalles refiérase a la documentación de la librería:
<br> http://www.coppeliarobotics.com/helpFiles/en/remoteApiFunctionsPython.htm

In [ ]:
def connect(port):
# Establece la conexión a CoppeliaSim
# port debe coincidir con el puerto de conexión en VREP
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID


In [ ]:
# Conectarse al servidor y obtener manejadores
clientID = connect(19999)

retCode,Dummy=sim.simxGetObjectHandle(clientID,'Dummy',sim.simx_opmode_blocking)
retCode,joint1=sim.simxGetObjectHandle(clientID,'joint1',sim.simx_opmode_blocking)
retCode,joint2=sim.simxGetObjectHandle(clientID,'joint2',sim.simx_opmode_blocking)
print(Dummy, joint1, joint2)

<h3> 1. Obtenemos la cinemática directa</h3>
Conocidos los ángulos de las articulaciones, podemos conocer la posición final del actuador.

In [ ]:
# Comenzamos desde la descripción de Denavit-Hartenberg
#      theta     |      d      |      a      |    alpha
# ----------------------------------------------------------
#       q1             0             0.3            0
#       q2             0             0.2            0

In [ ]:
# Definimos una función para construir las matrices de transformación
# en forma simbóĺica a partir de los parámetros D-H

def symTfromDH(theta, d, a, alpha):
    # theta y alpha en radianes
    # d y a en metros
    Rz = sp.Matrix([[sp.cos(theta), -sp.sin(theta), 0, 0],
                   [sp.sin(theta), sp.cos(theta), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    tz = sp.Matrix([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, d],
                   [0, 0, 0, 1]])
    ta = sp.Matrix([[1, 0, 0, a],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    Rx = sp.Matrix([[1, 0, 0, 0],
                   [0, sp.cos(alpha), -sp.sin(alpha), 0],
                   [0, sp.sin(alpha), sp.cos(alpha), 0],
                   [0, 0, 0, 1]])
    T = Rz*tz*ta*Rx
    return T

In [ ]:
# La matriz de transformación desde la base al dummy es
q1 = sp.symbols('q1')
q2 = sp.symbols('q2')

T = sp.simplify(symTfromDH(q1, 0, 0.3, 0)* symTfromDH(q2, 0, 0.2, 0))
T

<h3> 2. Situamos el robot en un punto mediante cinemática inversa</h3>

In [ ]:
# definimos las coordenadas de destino
a = 0.3
b = 0

# definimos las ecuaciones a resolver
ec1, ec2 = T[3]-a, T[7]-b
(ec1, ec2)

In [ ]:
# ahora resolvemos la ecuación utilizando nsolve()
try:
    q = sp.nsolve((ec1, ec2),(q1,q2),(1,-1), prec=6)
except:
    print("no se pudo calcular")
    q = [0, 0, 0]
q

In [ ]:
# enviamos los ángulos al robot
retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_oneshot)

<h3> 3. Y calculamos la cinemática diferencial en torno al punto de interés...</h3>